In [ ]:
# Full Tokenizer By Isaac Angulo Gomez 
pip install nltk

In [2]:
import nltk, ssl
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

for res in ["punkt", "punkt_tab"]:
    try:
        nltk.data.find(f"tokenizers/{res}")
    except LookupError:
        nltk.download(res)

from nltk.tokenize import sent_tokenize

import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from transformers import AutoTokenizer
tok_hf = AutoTokenizer.from_pretrained("bert-base-uncased")
tok_hf.save_pretrained("./bert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

('./bert-base-uncased/tokenizer_config.json',
 './bert-base-uncased/special_tokens_map.json',
 './bert-base-uncased/vocab.txt',
 './bert-base-uncased/added_tokens.json',
 './bert-base-uncased/tokenizer.json')

In [5]:
import sys, os
sys.path.append(os.getcwd())
from Tokenizer import FullTokenizer

30522


In [6]:
#Tokenizer Object
my_tok = FullTokenizer("./bert-base-uncased/vocab.txt", do_lower_case = True)

In [7]:
def make_tokenized_documents(raw_docs, tokenizer):
    out = []

    for doc in raw_docs:
        sents = sent_tokenize(doc)
        sent_ids = []

        for s in sents:
            toks = tokenizer.tokenize(s)
            ids = tokenizer.convert_tokens_to_ids(toks)

            if ids:
                sent_ids.append(ids)

        if sent_ids:
            out.append(sent_ids)

    return out

In [8]:
special_ids = {
    "cls": tok_hf.cls_token_id,
    "sep": tok_hf.sep_token_id,
    "pad": tok_hf.pad_token_id if tok_hf.pad_token_id is not None else 0,
    "mask": tok_hf.mask_token_id,
}

In [9]:
vocab_size = tok_hf.vocab_size

In [10]:
import sys, os
sys.path.append(os.getcwd())

from bert_pretrain_data import build_pretraining_instances, BertPretrainDataset, bert_collate_fn

In [11]:
raw_docs = [
    "The quick brown fox jumped over the lazy dog. It then ran away.",
    "Transformers are powerful models. BERT is pre-trained with MLM and NSP."
]

tokenized_documents = make_tokenized_documents(raw_docs, my_tok)

In [12]:
instances = build_pretraining_instances(tokenized_documents, special_ids = special_ids, vocab_size = vocab_size, max_seq_len = 128, short_seq_prob = 0.1, nsp_prob = 0.5, mask_prob = 0.15, seed = 123)

In [13]:
#Examples of first 3 instances
for i, inst in enumerate(instances[:3]):
    print(f"---- Instance {i} ----")
    print("NSP label:", "IsNext" if inst.nsp_label == 1 else "NotNext")
    print("len(input_ids):", len(inst.input_ids))

    # reconstruct token strings from IDs (for easier human reading)
    tokens = [tok_hf.convert_ids_to_tokens([tid])[0] for tid in inst.input_ids]
    masked_tokens = []
    for t, label in zip(tokens, inst.mlm_labels):
        if label != -100:
            # mark masked tokens visibly
            masked_tokens.append(f"*{t}*")
        else:
            masked_tokens.append(t)
    print("Tokens:", " ".join(masked_tokens))
    print()

---- Instance 0 ----
NSP label: IsNext
len(input_ids): 18
Tokens: [CLS] the *payton* brown fox *[MASK]* over the lazy dog . [SEP] it then ran away . [SEP]

---- Instance 1 ----
NSP label: NotNext
len(input_ids): 25
Tokens: [CLS] it then ran away . [SEP] *[MASK]* are *[MASK]* models . *[MASK]* is pre - trained with ml ##m and ns ##p . [SEP]

---- Instance 2 ----
NSP label: NotNext
len(input_ids): 35
Tokens: [CLS] transformers *[MASK]* powerful models . bert is pre - trained with ml ##m and ns ##p *[MASK]* [SEP] the *[MASK]* brown fox jumped over the lazy dog . *[MASK]* then ran *[MASK]* . [SEP]



In [14]:
vocab_file = "bert-base-uncased/vocab.txt"  # path to downloaded vocab
tokenizer = FullTokenizer(vocab_file=vocab_file, do_lower_case=True)

In [15]:
#Check tokenizer using vocab
print(tokenizer.tokenize("The quick brown fox jumped over the lazy dog."))
print(tokenizer.convert_tokens_to_ids(["[CLS]", "the", "quick", "brown", "[SEP]"]))
print(len(tokenizer.vocab))

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', '.']
[101, 1996, 4248, 2829, 102]
30522


In [16]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00


In [18]:
# Full training and validation by Nicolas Mendoza

# NOTE!! Before running this cell please rename the directory "bert-base-uncased" to "bert-base-uncased-t"
# The model cannot be imported if there is another directory called "bert-base-uncased" so just change it to this ^^

from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from Tokenizer import build_inputs_from_texts
import evaluate
import torch
import os

os.environ["WANDB_DISABLED"] = "true"

dataset = load_dataset("glue", "rte")

# Using our tokenizer
def encode_batch(batch):
    encoded = build_inputs_from_texts(
        tokenizer,
        batch["sentence1"],
        batch["sentence2"],
        max_len=256
    )
    return {
        "input_ids": encoded["input_ids"],
        "token_type_ids": encoded["token_type_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": batch["label"]
    }

# Prep data
encoded = dataset.map(encode_batch)
encoded.set_format(type="torch",
                   columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

# Get pretrained bert model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

metric = evaluate.load("glue", "rte")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return metric.compute(predictions=preds, references=labels)

# Args for training
args = TrainingArguments(
    output_dir="./bert_output",
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=200,
    save_total_limit=1,
)

# Training
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded["train"],
    eval_dataset=encoded["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate()
print(results)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
200,0.629800
400,0.417100


{'eval_loss': 0.7153940200805664, 'eval_accuracy': 0.6678700361010831, 'eval_runtime': 3.8422, 'eval_samples_per_second': 72.094, 'eval_steps_per_second': 9.109, 'epoch': 3.0}
